In [ ]:
import csv
import os
import sys

# Get results from the results file
results = {}
with open("results.csv", newline='') as results_csv:
    csv_reader = csv.reader(results_csv)
    # Get the header
    header = next(csv_reader)
    # Now get the results
    row = next(csv_reader)
    for i in range(0, len(row)):
        results[header[i]] = row[i]
# Download files in results
result_files = {}
for key, val in results.items():
    if val.startswith('gs://'):
        # Attempt to download with gsutil
        os.makedirs(f'carrot_downloads/{key}', exist_ok=True)
        filepath = f'carrot_downloads/{key.replace(" ", "_")}/{val[val.rfind("/")+1:]}'
        download_status = os.system(f'gsutil cp {val} {filepath} >/dev/null 2>&1')
        # If it failed, print an error message and exit
        if download_status != 0:
            sys.exit(f"gsutil terminated with an non-zero exit code when attempting to download {val}")
        # If it succeeded, add it to our file list
        result_files[key] = filepath

In [ ]:
import csv

# Load data and comparison files
data = [[], []]
with open(result_files["example data file"]) as data_file:
    data_csv = csv.reader(data_file)
    for row in data_csv:
        if len(row) > 1:
            data[0].append(float(row[0]))
            data[1].append(float(row[1]))

comparison = [[], []]
with open(result_files["example comparison file"]) as comparison_file:
    comparison_csv = csv.reader(comparison_file)
    for row in comparison_csv:
        if len(row) > 1:
            comparison[0].append(float(row[0]))
            comparison[1].append(float(row[1]))

# Prepare the default data
default = [[], []]
for i in range(0, 21, 1):
    default[0].append(i/10.0)
    default[1].append(pow(2, i/10.0))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

fig, ax = plt.subplots()
# Plot data first
ax.scatter(data[0], data[1], c='tab:blue', label="data")
# Then comparison
ax.scatter(comparison[0], comparison[1], c='tab:green', label="diff")
# Finally the default that was compared against
ax.scatter(default[0], default[1], c='tab:red', label="default")

ax.legend()
ax.grid(True)

y_vals = data[1] + comparison[1] + default[1]
y_max = math.ceil(max(y_vals))
plt.xticks(np.arange(0, 2.0, 0.2))
plt.yticks(np.arange(0, y_max, y_max/10.0))

plt.show()